In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

# =========================================================
# 1. LOAD DATA SINGAPORE (WIDE IMF)
# =========================================================
# Sesuaikan path jika file kamu tidak di folder "data"
df = pd.read_csv("data/DataSingapore.csv")

# Drop kolom yang tidak dipakai (sama seperti Jepang)
df = df.drop(columns=["DATASET", "SERIES_CODE", "OBS_MEASURE"])

# =========================================================
# 2. Pastikan kolom tahun 2024–2030 ada
# =========================================================
forecast_years = list(range(2024, 2031))  # 2024–2030

for y in forecast_years:
    col = str(y)
    if col not in df.columns:
        df[col] = np.nan

# =========================================================
# 3. Ambil kolom historis (2000–2023)
#    -> Sesuaikan kalau mau 2023 dianggap forecast/estimate
# =========================================================
year_hist = list(range(2000, 2024))  # 2000–2023
year_cols_hist = [str(y) for y in year_hist]

# =========================================================
# 4. ETS FORECAST 2024–2030
# =========================================================
all_fc = np.zeros((len(df), len(forecast_years)))

for i, (_, row) in enumerate(df.iterrows()):
    series = row[year_cols_hist].astype(float).dropna().values

    # Kalau data historis terlalu pendek, skip
    if len(series) < 5:
        continue

    model = ExponentialSmoothing(
        series,
        trend="add",
        seasonal=None,
        initialization_method="estimated"
    )
    fit = model.fit(optimized=True)

    fc = fit.forecast(len(forecast_years))
    all_fc[i, :] = fc

# Masukkan hasil forecast ke kolom tahun 2024–2030
for idx, y in enumerate(forecast_years):
    df[str(y)] = all_fc[:, idx]

# =========================================================
# 5. Wide → Long
# =========================================================
all_year_cols = [str(y) for y in (year_hist + forecast_years)]
id_vars = [c for c in df.columns if c not in all_year_cols]

long_df = df.melt(
    id_vars=id_vars,
    value_vars=all_year_cols,
    var_name="Year",
    value_name="Value"
)

long_df["Year"] = long_df["Year"].astype(int)
long_df["Value"] = pd.to_numeric(long_df["Value"], errors="coerce")

# =========================================================
# 6. Tambah kolom Type: Actual / Forecast
# =========================================================
# Versi simple: 2000–2023 = Actual, 2024–2030 = Forecast
long_df["Type"] = np.where(long_df["Year"] <= 2023, "Actual", "Forecast")

# Kalau mau 2023 sebagai Forecast, ganti baris di atas dengan:
# long_df["Type"] = np.where(long_df["Year"] <= 2022, "Actual", "Forecast")

# =========================================================
# 7. Simpan FINAL
# =========================================================
output_path = "data/DataSingaporeForecast.csv"
long_df.to_csv(output_path, index=False, float_format="%.2f")

long_df


,COUNTRY,INDICATOR,FREQUENCY,SCALE,Year,Value,Type
0,Singapore,"Gross domestic product (GDP), Constant prices,...",Annual,Units,2000,9.04,Actual
1,Singapore,"All Items, Consumer price index (CPI), Period ...",Annual,Units,2000,1.35,Actual
2,Singapore,"Net lending (+) / net borrowing (-), General g...",Annual,Units,2000,4.59,Actual
3,Singapore,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2000,2.06,Actual
4,Singapore,"Gross domestic product (GDP), Constant prices,...",Annual,Units,2000,7.17,Actual
...,...,...,...,...,...,...,...
212,Singapore,"Net lending (+) / net borrowing (-), General g...",Annual,Units,2030,2.22,Forecast
213,Singapore,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2030,5.19,Forecast
214,Singapore,"Gross domestic product (GDP), Constant prices,...",Annual,Units,2030,1.19,Forecast
215,Singapore,Unemployment rate,Annual,Units,2030,1.70,Forecast
